In [1]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [2]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [3]:
openai=OpenAI()
model='gpt-4o'

In [4]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an Ryzen5 processor. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [5]:
def user_prompt(python):
    prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    prompt += python
    return prompt

In [6]:
def message_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt(python)}
    ]

In [ ]:
def write_output(cpp):
    